In [2]:
import os
import pydicom
from pydicom.misc import is_dicom
import matplotlib.pyplot as plt
import numpy as np
import SimpleITK as sitk

In [ ]:
def load_dicom_images(directory, max_slices=None):
    slices = []
    for filename in sorted(os.listdir(directory)):
        if filename.endswith(".dcm"):
            path = os.path.join(directory, filename)
            dicom = pydicom.dcmread(path)
            slices.append(dicom)
            if max_slices and len(slices) >= max_slices:
                break
    # Ordenar as fatias por posição do paciente
    slices.sort(key=lambda x: x.ImagePositionPatient[2])
    return slices

def create_3d_image(slices):
    # Extrair arrays de pixel e empilhá-los
    pixel_arrays = [s.pixel_array for s in slices]
    image_3d = np.stack(pixel_arrays, axis=-1)
    # Converter para imagem ITK
    itk_image = sitk.GetImageFromArray(image_3d)
    return itk_image

def show_slices(image_array, num_slices=165, cols=11):
    # Calcular número de linhas necessárias
    rows = num_slices // cols + (num_slices % cols > 0)
    fig, axes = plt.subplots(rows, cols, figsize=(20, 20))

    for i in range(num_slices):
        row = i // cols
        col = i % cols
        axes[row, col].imshow(image_array[:, :, i], cmap='gray')
        axes[row, col].axis('off')
        axes[row, col].set_title(f'Slice {i}')

    # Remover subplots extras se o número de fatias for menor que o grid total
    for j in range(num_slices, rows * cols):
        fig.delaxes(axes.flatten()[j])

    plt.tight_layout()
    plt.show()

def correct_orientation(itk_image):
    # Obter a matriz de transformação atual
    origin = itk_image.GetOrigin()
    direction = itk_image.GetDirection()
    spacing = itk_image.GetSpacing()
    
    # Alterar a direção da imagem para que os eixos estejam orientados corretamente
    # Isso depende da orientação específica dos dados DICOM
    direction_corrected = [1, 0, 0, 0, 1, 0, 0, 0, -1]  # Exemplo para inverter o eixo Z
    
    itk_image.SetDirection(direction_corrected)
    
    # Atualizar a origem da imagem se necessário
    # Exemplo: modificar a origem para compensar a mudança na orientação
    origin_corrected = (origin[0], origin[1], origin[2] + (len(slices) - 1) * spacing[2])
    itk_image.SetOrigin(origin_corrected)
    
    return itk_image

# Caminho p DICOM
dicom_directory = "C:/Users/bruno/Desktop/IANS/mri_exemplo/DICOM/I60175"

# Carregar  DICOM
slices = load_dicom_images(dicom_directory, max_slices=166)  # Limitar a 166 imagens
image_3d = create_3d_image(slices)

# Corrigir a orientação da imagem
image_3d_corrected = correct_orientation(image_3d)

# Converter a imagem segmentada para um array NumPy para visualização
img_array = sitk.GetArrayFromImage(image_3d_corrected)

# Visualizar os cortes 2D da imagem segmentada (somente o cérebro)
show_slices(img_array, num_slices=166, cols=11)

# Salvar a imagem 3D segmentada como um arquivo NIfTI
sitk.WriteImage(image_3d, "C:/Users/bruno/Desktop/IANS/mri_exemplo/NIFTL/output_brain_2_oriented.nii.gz")

In [3]:
def print_image(image):

    # Converte a imagem para um numpy array
    image_array = image.numpy()

    # Obtém o índice central para cada dimensão
    axial_index = image_array.shape[0] // 2  # Corte axial
    coronal_index = image_array.shape[1] // 2  # Corte coronal
    sagital_index = image_array.shape[2] // 2  # Corte sagital

    # Cria um gráfico com 3 subgráficos
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    # Exibe o corte axial
    axes[0].imshow(image_array[:, :, axial_index], cmap='gray')
    axes[0].set_title('Corte Axial')
    axes[0].axis('off')

    # Exibe o corte coronal
    axes[1].imshow(image_array[:, coronal_index, :], cmap='gray')
    axes[1].set_title('Corte Coronal')
    axes[1].axis('off')

    # Exibe o corte sagital
    axes[2].imshow(image_array[sagital_index, :, :], cmap='gray')
    axes[2].set_title('Corte Sagital')
    axes[2].axis('off')

    # Ajusta o layout e exibe os cortes
    plt.tight_layout()
    plt.show()

In [ ]:
import os

DIR_BASE = os.path.abspath('/mnt/d/ADNI/ADNI1')
DIR_RAW = os.path.join(DIR_BASE, 'ADNI_nii_raw')
DIR_OUTPUT = os.path.join(DIR_BASE, 'ADNI_nii_processed')

raw = [(item) for item in os.listdir(DIR_RAW)]
processed = [(item) for item in os.listdir(DIR_OUTPUT)]

for item in processed:
    if item in raw:
        raw.remove(item)

for item in raw:
    print(item)

In [ ]:
import ants
import os
import nibabel as nib
cont = 0

print_image(ants.image_read(os.path.join("/mnt/d/ADNI/ADNI1/mni_icbm152_nlin_asym_09c/mni_icbm152_t1_tal_nlin_asym_09c_mask.nii")))

print("PROCESSADA SEM AJUSTE MÁSCARA")

for item in os.listdir("/mnt/d/ADNI/ADNI1/ADNI_faltou_processed"):
    if cont < 15:
        print_image(ants.image_read(os.path.join("/mnt/d/ADNI/ADNI1/ADNI_faltou_processed", item)))

print("PROCESSADA COM AJUSTE MÁSCARA")

for item in os.listdir("/mnt/d/ADNI/ADNI1/ADNI_faltou_mask_adjust"):
    if cont < 30:
        print_image(ants.image_read(os.path.join("/mnt/d/ADNI/ADNI1/ADNI_faltou_mask_adjust", item)))



In [ ]:
import pandas as pd

# Carregar o arquivo CSV
csv_file = '/mnt/c/Users/bruno/Desktop/IANS/alzheimer_project/codes/mri/ADNI1_Screening_10_04_2024.csv'  # Substitua pelo nome do seu arquivo
df = pd.read_csv(csv_file)

# Contar quantas vezes cada 'Subject' aparece
subject_counts = df['Subject'].value_counts()

# Ajustar para que o pandas mostre todas as linhas
pd.set_option('display.max_rows', None)

# Imprimir a quantidade de vezes que cada 'Subject' aparece
print(subject_counts)


In [15]:
import os
import pandas as pd
from collections import Counter

# Contador para registrar as ocorrências
cn, mci, ad = 0, 0, 0

# Caminho para o CSV
csv = '/mnt/c/Users/bruno/Desktop/IANS/alzheimer_project/codes/mri/ADNI1_Screening_10_04_2024.csv'
df = pd.read_csv(csv)

# Diretorio
directory = os.listdir('/mnt/d/ADNI/ADNI1/ADNI_nii_processed')

# Converte a coluna 'Subject' em uma lista
subjects = {'cn': [], 'mci': [], 'ad': []}

for subject in directory:
    name = subject.rsplit('.', 2)[0]
    line = df[df['Subject'] == name]

    group = line['Group'].iloc[0]
    
    if group == 'CN':
        subjects['cn'].append(name)
    elif group == 'MCI':
        subjects['mci'].append(name)
    elif group == 'AD':
        subjects['ad'].append(name)

# Conta as ocorrências dos arquivos na lista dir
print(subjects['cn'])
print(subjects['mci'])
print(subjects['ad'])

print(len(subjects['cn']))
print(len(subjects['mci']))
print(len(subjects['ad']))



['002_S_0295', '002_S_0413', '002_S_0559', '002_S_0685', '002_S_1261', '002_S_1280', '005_S_0223', '005_S_0602', '006_S_0484', '006_S_0498', '006_S_0731', '007_S_0068', '007_S_0070', '007_S_1206', '007_S_1222', '014_S_0519', '014_S_0520', '014_S_0548', '014_S_0558', '021_S_0159', '021_S_0337', '021_S_0647', '021_S_0984', '024_S_0985', '024_S_1063', '027_S_0074', '027_S_0118', '027_S_0120', '027_S_0403', '029_S_0824', '029_S_0843', '029_S_0845', '031_S_0618', '033_S_0516', '033_S_0734', '033_S_0741', '033_S_0920', '033_S_0923', '033_S_1086', '037_S_0303', '037_S_0327', '037_S_0454', '037_S_0467', '052_S_0951', '052_S_1250', '052_S_1251', '094_S_0489', '094_S_0526', '094_S_0692', '094_S_0711', '094_S_1241', '094_S_1267', '098_S_0171', '098_S_0172', '098_S_0896', '116_S_0360', '116_S_0382', '116_S_0648', '116_S_0657', '116_S_1232', '116_S_1249', '127_S_0259', '127_S_0260', '127_S_0622', '127_S_0684', '128_S_0229', '128_S_0230', '128_S_0245', '128_S_0272', '128_S_0500', '128_S_0522', '128_

In [16]:
import random

backup = subjects

#dividir pastas

def distributor(subjects, new_dir, percentage):
    # Distribuindo CN
    for i in range(int(percentage * len(subjects['cn']))):
        id = random.choice(subjects['cn'])
        new_dir['cn'].append(id)
        subjects['cn'].remove(id)

    # Distribuindo MCI
    for i in range(int(percentage * len(subjects['mci']))):
        id = random.choice(subjects['mci'])
        new_dir['mci'].append(id)
        subjects['mci'].remove(id)

    # Distribuindo AD
    for i in range(int(percentage * len(subjects['ad']))):
        id = random.choice(subjects['ad'])
        new_dir['ad'].append(id)
        subjects['ad'].remove(id)

#treino (70)
train = {'cn': [], 'mci': [], 'ad': []}

distributor(subjects, train, 0.7)   

#validação (20)
validation = {'cn': [], 'mci': [], 'ad': []}

distributor(subjects, validation, 0.2)   

#teste (10)
test = {'cn': [], 'mci': [], 'ad': []}

distributor(subjects, test, 0.1)   

['002_S_0295', '002_S_0413', '002_S_0559', '002_S_0685', '002_S_1261', '002_S_1280', '005_S_0223', '005_S_0602', '006_S_0484', '006_S_0498', '006_S_0731', '007_S_0068', '007_S_0070', '007_S_1206', '007_S_1222', '014_S_0519', '014_S_0520', '014_S_0548', '014_S_0558', '021_S_0159', '021_S_0337', '021_S_0647', '021_S_0984', '024_S_0985', '024_S_1063', '027_S_0074', '027_S_0118', '027_S_0120', '027_S_0403', '029_S_0824', '029_S_0843', '029_S_0845', '031_S_0618', '033_S_0516', '033_S_0734', '033_S_0741', '033_S_0920', '033_S_0923', '033_S_1086', '037_S_0303', '037_S_0327', '037_S_0454', '037_S_0467', '052_S_0951', '052_S_1250', '052_S_1251', '094_S_0489', '094_S_0526', '094_S_0692', '094_S_0711', '094_S_1241', '094_S_1267', '098_S_0171', '098_S_0172', '098_S_0896', '116_S_0360', '116_S_0382', '116_S_0648', '116_S_0657', '116_S_1232', '116_S_1249', '127_S_0259', '127_S_0260', '127_S_0622', '127_S_0684', '128_S_0229', '128_S_0230', '128_S_0245', '128_S_0272', '128_S_0500', '128_S_0522', '128_

In [18]:
#INFORMAÇÕES

print("FULL DATASET:")
print(f"CN: {len(backup['cn'])}   MCI: {len(backup['mci'])}   AD: {len(backup['ad'])}")
print(f"{backup['cn']}\n{backup['mci']}\n{backup['ad']}\n\n")

print("TRAIN DATASET:")
print(f"CN: {len(train['cn'])}   MCI: {len(train['mci'])}   AD: {len(train['ad'])}")
print(f"{train['cn']}\n{train['mci']}\n{train['ad']}\n\n")

print("VALIDATION DATASET:")
print(f"CN: {len(validation['cn'])}   MCI: {len(validation['mci'])}   AD: {len(validation['ad'])}")
print(f"{validation['cn']}\n{validation['mci']}\n{validation['ad']}\n\n")

print("TEST DATASET")
print(f"CN: {len(test['cn'])}   MCI: {len(test['mci'])}   AD: {len(test['ad'])}")
print(f"{test['cn']}\n{test['mci']}\n{test['ad']}\n\n")

FULL DATASET:
CN: 21   MCI: 40   AD: 18
['002_S_0559', '024_S_1063', '033_S_0734', '033_S_0741', '033_S_0920', '033_S_0923', '037_S_0303', '037_S_0454', '037_S_0467', '094_S_0489', '094_S_0692', '094_S_1241', '094_S_1267', '116_S_0382', '127_S_0259', '127_S_0260', '127_S_0622', '131_S_0123', '136_S_0184', '137_S_0283', '137_S_0686']
['005_S_0546', '006_S_0675', '007_S_0041', '007_S_0101', '014_S_0658', '021_S_0178', '021_S_0231', '021_S_0273', '021_S_0276', '027_S_0256', '027_S_0408', '027_S_0485', '027_S_1387', '029_S_1218', '033_S_0514', '037_S_0501', '037_S_1078', '052_S_0989', '094_S_0921', '094_S_1188', '098_S_0269', '116_S_0649', '116_S_0834', '116_S_0890', '127_S_0112', '127_S_0397', '127_S_1032', '127_S_1427', '128_S_0225', '128_S_0227', '128_S_0258', '128_S_0715', '129_S_1204', '129_S_1246', '131_S_1389', '136_S_0195', '136_S_0695', '136_S_0873', '137_S_0481', '137_S_1414']
['005_S_0221', '006_S_0653', '014_S_0357', '014_S_1095', '021_S_0642', '027_S_1254', '029_S_1184', '031_